# Update an existing Tag

In [1]:
from django.test import RequestFactory

from organizer.models import Tag
from organizer.serializers import StartupSerializer, TagSerializer

In [2]:
fake_request = RequestFactory().get("/api/v1/tag/")

In [3]:
tag, created = Tag.objects.get_or_create(
    slug="django",
    defaults={
        "name": "django",
    },
)

In [4]:
s_tag = TagSerializer(
    tag,  # the existing object
    data={
        "name": "django!",
    },
    context={"request": fake_request},
)

In [5]:
s_tag.is_valid()

True

In [6]:
s_tag.save()

<Tag: django!>

In [7]:
tag.refresh_from_db()

In [8]:
tag.name

'django!'

In [9]:
tag.slug

'django'

## Partial Updates and Ignored Fields

In [10]:
s_tag = TagSerializer(
    tag,  # the existing object
    data={
        "slug": "hendrix",
        "ignored": "this won't cause problems"
    },
    context={"request": fake_request},
)

In [11]:
s_tag.is_valid()

False

In [12]:
s_tag.errors

{'name': [ErrorDetail(string='This field is required.', code='required')]}

In [13]:
s_tag = TagSerializer(
    tag,  # the existing object
    data={
        "slug": "hendrix",
        "ignored": "this won't cause problems"
    },
    partial=True,
    context={"request": fake_request},
)

In [14]:
s_tag.is_valid()

True

In [15]:
s_tag.save()

<Tag: django!>

In [16]:
tag.refresh_from_db()

In [17]:
tag.slug

'django'

In [18]:
# the AutoSlugField tells other parts of
# Django and third-party apps to not update this field
# and so the Serializer silently leaves it alone!
Tag._meta.get_field("slug").editable

False

In [19]:
s_tag  # note the "read_only" value in the SlugField below

TagSerializer(<Tag: django!>, context={'request': <WSGIRequest: GET '/api/v1/tag/'>}, data={'slug': 'hendrix', 'ignored': "this won't cause problems"}, partial=True):
    url = HyperlinkedIdentityField(lookup_field='slug', view_name='api-tag-detail')
    name = CharField(max_length=31, validators=[<UniqueValidator(queryset=Tag.objects.all())>])
    slug = SlugField(help_text='A label for URL config.', read_only=True)

## Notebook Cleanup

In [20]:
tag.name = 'django'
tag.save()